In [1]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import pynq.lib.dma
import time
import pynq
import pandas as pd

In [2]:
weight_0 = np.load('weights_final/weight_0.npy')
weight_1 = np.load('weights_final/weight_1.npy')
weight_2 = np.load('weights_final/weight_2.npy')
weight_3 = np.load('weights_final/weight_3.npy')
weight_4 = np.load('weights_final/weight_4.npy')
bias_0 = np.load('weights_final/bias_0.npy')
bias_1 = np.load('weights_final/bias_1.npy')
bias_2 = np.load('weights_final/bias_2.npy')
bias_3 = np.load('weights_final/bias_3.npy')
bias_4 = np.load('weights_final/bias_4.npy')
weight_0_pos = np.load('weights_3dance_pos/weight_0.npy')
weight_1_pos = np.load('weights_3dance_pos/weight_1.npy')
weight_2_pos = np.load('weights_3dance_pos/weight_2.npy')
weight_3_pos = np.load('weights_3dance_pos/weight_3.npy')
bias_0_pos = np.load('weights_3dance_pos/bias_0.npy')
bias_1_pos = np.load('weights_3dance_pos/bias_1.npy')
bias_2_pos = np.load('weights_3dance_pos/bias_2.npy')
bias_3_pos = np.load('weights_3dance_pos/bias_3.npy')
scaler_mean = np.load('weights_final/mean.npy')
scaler_scale = np.load('weights_final/scale.npy')
scaler_mean_pos = np.load('weights_3dance_pos/scaler_mean.npy')
scaler_scale_pos = np.load('weights_3dance_pos/scaler_scale.npy')

In [3]:
df1 = pd.read_csv("dataset_3dance_pos/dancedata.csv")
y_training_pos = pd.Series(df1.LABEL).transpose()
X_training_pos = df1.drop(['Unnamed: 0', 'LABEL'], axis=1).values
X_training_pos = (X_training_pos - scaler_mean_pos) / scaler_scale_pos

In [4]:
df = pd.read_csv("dataset_final/dancedata.csv")
y_training = pd.Series(df.LABEL).transpose()
X_training = df.drop(['Unnamed: 0', 'LABEL'], axis=1).values
X_training = (X_training - scaler_mean) / scaler_scale

In [5]:
overlay = Overlay('final.bit')   # load bitstream inside FPGA
dma = overlay.axi_dma_0    

In [6]:
choose_model = allocate(shape=(1,), dtype=np.float32)
input_buffer0 = allocate(shape=(100,), dtype=np.float32)
input_buffer1 = allocate(shape=(128,), dtype=np.float32)
input_buffer2 = allocate(shape=(12,), dtype=np.float32)
input_buffer3 = allocate(shape=(32,), dtype=np.float32)
input_buffer4 = allocate(shape=(3,), dtype=np.float32)
output_buffer0 = allocate(shape=(12,), dtype=np.float32)
output_buffer1 = allocate(shape=(3,), dtype=np.float32)

In [7]:
##load dance weights
for i in range(128):
    for k in range(100):
        input_buffer0[k] = weight_0[k][i]
    dma.sendchannel.transfer(input_buffer0)
    dma.sendchannel.wait()
for i in range(128):
    for k in range(128):
        input_buffer1[k] = weight_1[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
for i in range(128):
    for k in range(128):
        input_buffer1[k] = weight_2[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
for i in range(128):
    for k in range(128):
        input_buffer1[k] = weight_3[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
for i in range(12):
    for k in range(128):
        input_buffer1[k] = weight_4[k][i]
    dma.sendchannel.transfer(input_buffer1)
    dma.sendchannel.wait()
##bias
for k in range(128):
    input_buffer1[k] = bias_0[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(128):
    input_buffer1[k] = bias_1[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(128):
    input_buffer1[k] = bias_2[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(128):
    input_buffer1[k] = bias_3[k]
dma.sendchannel.transfer(input_buffer1)
dma.sendchannel.wait()
for k in range(12):
    input_buffer2[k] = bias_4[k]
dma.sendchannel.transfer(input_buffer2)
dma.sendchannel.wait()

In [8]:
##load position weights
for i in range(32):
    for k in range(100):
        input_buffer0[k] = weight_0_pos[k][i]
    dma.sendchannel.transfer(input_buffer0)
    dma.sendchannel.wait()
for i in range(32):
    for k in range(32):
        input_buffer3[k] = weight_1_pos[k][i]
    dma.sendchannel.transfer(input_buffer3)
    dma.sendchannel.wait()
for i in range(32):
    for k in range(32):
        input_buffer3[k] = weight_2_pos[k][i]
    dma.sendchannel.transfer(input_buffer3)
    dma.sendchannel.wait()
for i in range(3):
    for k in range(32):
        input_buffer3[k] = weight_3_pos[k][i]
    dma.sendchannel.transfer(input_buffer3)
    dma.sendchannel.wait()
##bias
for k in range(32):
    input_buffer3[k] = bias_0_pos[k]
dma.sendchannel.transfer(input_buffer3)
dma.sendchannel.wait()
for k in range(32):
    input_buffer3[k] = bias_1_pos[k]
dma.sendchannel.transfer(input_buffer3)
dma.sendchannel.wait()
for k in range(32):
    input_buffer3[k] = bias_2_pos[k]
dma.sendchannel.transfer(input_buffer3)
dma.sendchannel.wait()
for k in range(3):
    input_buffer4[k] = bias_3_pos[k]
dma.sendchannel.transfer(input_buffer4)
dma.sendchannel.wait()

In [9]:
def fpga_evaluate_dance(test):
    choose_model[0] = 0
    dma.sendchannel.transfer(choose_model)
    dma.sendchannel.wait()
    for j in range(100):
        input_buffer0[j] = test[j];
    dma.sendchannel.transfer(input_buffer0)
    dma.recvchannel.transfer(output_buffer0)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    return output_buffer0

In [10]:
def fpga_evaluate_pos(test):
    choose_model[0] = 1
    dma.sendchannel.transfer(choose_model)
    dma.sendchannel.wait()
    for j in range(100):
        input_buffer0[j] = test[j];
    dma.sendchannel.transfer(input_buffer0)
    dma.recvchannel.transfer(output_buffer1)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    return output_buffer1

In [11]:
X_training.shape

(8351, 100)

In [20]:
fpga_res = np.zeros(12)
for i in range(X_training.shape[0]):
    fpga_res = np.row_stack((fpga_res, fpga_evaluate_dance(X_training[i])))
fpga_res = fpga_res[1:]

In [21]:
fpga_res = np.argmax(fpga_res, axis=-1)

In [22]:
labels = np.array(['dab', 'elbowkick', 'gun', 'hair', 'left', 'listen', 'pointhigh', 'right', 'sidepump', 'special', 'stationary', 'wipetable'])
decoded_predictions = labels[fpga_res]

In [23]:
from IPython.display import display
dataset = pd.DataFrame({'FPGA predictions': decoded_predictions, 'Actual test label': y_training}).reset_index(drop=True)
pd.set_option('display.max_rows', 300)
pd.set_option('display.height', 300)
display(dataset)

,Actual test label,FPGA predictions
0,dab,stationary
1,dab,left
2,dab,left
3,dab,stationary
4,dab,dab
5,dab,dab
6,dab,sidepump
7,dab,dab
8,dab,dab
9,dab,dab


In [24]:
fpga_pos_res = np.zeros(3)
for i in range(X_training_pos.shape[0]):
    fpga_pos_res = np.row_stack((fpga_pos_res, fpga_evaluate_pos(X_training_pos[i])))
fpga_pos_res = fpga_pos_res[1:]

In [25]:
fpga_pos_res = np.argmax(fpga_pos_res, axis=-1)

In [26]:
labels_pos = np.array(['left', 'right', 'stationary'])
decoded_pos_predictions = labels_pos[fpga_pos_res]

In [27]:
from IPython.display import display
dataset_pos = pd.DataFrame({'FPGA predictions': decoded_pos_predictions, 'Actual test label': y_training_pos}).reset_index(drop=True)
pd.set_option('display.max_rows', 300)
pd.set_option('display.height', 300)
display(dataset_pos)

,Actual test label,FPGA predictions
0,left,left
1,left,left
2,left,left
3,left,left
4,left,left
5,left,left
6,left,left
7,left,left
8,left,left
9,left,left
